In [2]:
import pandas as pd
import numpy as np
import folium, json ## desde terminal poner "pip install folium"
eventos = pd.read_csv("../Data/events.csv", low_memory=False)
eventos["timestamp"] = pd.to_datetime(eventos["timestamp"])

# Análisis de los usuarios de Brasil

# Cantidad de usuarios por país

In [4]:
usuarios = eventos.drop_duplicates(subset="person",keep="first",inplace=False)
usuarios = usuarios.loc[usuarios["country"] != "Unknown"]
usuarios = usuarios.loc[:,["person", "city", "region", "country"]]
usuarios["country"].value_counts()

Brazil            11008
United States       132
Argentina            16
Mozambique            6
France                5
Canada                5
Bangladesh            4
Netherlands           2
United Kingdom        2
Portugal              2
Morocco               2
Italy                 1
Ireland               1
Israel                1
Philippines           1
South Africa          1
French Guiana         1
Belize                1
Cape Verde            1
Saudi Arabia          1
India                 1
Sri Lanka             1
Turkey                1
Germany               1
Angola                1
Uruguay               1
China                 1
Venezuela             1
Colombia              1
Russia                1
Name: country, dtype: int64

### La cantidad de usuarios provenientes de Brasil es significativamente mayor a la de provenientes de otros paises, por lo que se analizará cómo se distribuyen los primeros a lo largo y a lo ancho del territorio brasilero.

## Distribución por región de los usuarios brasileros (cada 1 millon de habitantes)

In [17]:
usuariosDeBrasil = usuarios.loc[eventos["country"]=="Brazil"]
usuariosDeBrasil = usuariosDeBrasil.loc[:,["person","region"]]
usuariosDeBrasil = usuariosDeBrasil.loc[usuariosDeBrasil["region"] != "Unknown"]
usuariosDeBrasil = usuariosDeBrasil.groupby("region").agg(np.size)
usuariosDeBrasil = usuariosDeBrasil.reset_index()
usuariosDeBrasil.columns = ["Region", "Cantidad de usuarios"]
usuariosDeBrasil["Region"] = ["Acre", "Alagoas", "Amap", "Amazonas", "Bahia", "Cear", "Esprito Santo", "DF", "Gois", "Maranho", "Mato Grosso", "Mato Grosso do Sul", "Minas Gerais", "Par", "Paran", "Paraba", "Pernambuco", "Piau", "Rio Grande do Norte", "Rio Grande do Sul", "Rio de Janeiro", "Rondnia", "Roraima", "Santa Catarina", "So Paulo", "Sergipe", "Tocantins"]
habitantesPorEstado = pd.read_csv("estados_brasil_habitantes.csv")
usuariosDeBrasil["Cantidad de habitantes (en millones)"] = habitantesPorEstado["Poblacion"]
usuariosDeBrasil["Usuarios por millon de habitantes"] = usuariosDeBrasil["Cantidad de usuarios"] / usuariosDeBrasil["Cantidad de habitantes (en millones)"]
usuariosDeBrasil.sort_values("Usuarios por millon de habitantes", ascending=False)

,Region,Cantidad de usuarios,Cantidad de habitantes (en millones),Usuarios por millon de habitantes
24,So Paulo,3190,44.00,72.500000
7,DF,198,3.03,65.346535
20,Rio de Janeiro,969,16.70,58.023952
4,Bahia,732,15.34,47.718383
12,Minas Gerais,976,21.20,46.037736
18,Rio Grande do Norte,147,3.40,43.235294
6,Esprito Santo,168,3.90,43.076923
5,Cear,358,8.84,40.497738
16,Pernambuco,382,9.50,40.210526
26,Tocantins,58,1.50,38.666667


## Mapa coroplético según región

In [16]:
mapa = folium.Map(location=[-30, -40], zoom_start=3)

mapa.choropleth(
    geo_data=open("brasil_estados.geojson").read(),
    name="choropleth",
    data=usuariosDeBrasil,
    columns=["Region","Usuarios por millon de habitantes"],
    key_on="feature.properties.nome_uf",
    #threshold_scale=[3, 50, 100, 400, 1000, 3000],
    fill_color="YlOrBr",
    fill_opacity=0.9,
    line_opacity=0.5,
    legend_name="Usuarios por millon de habitantes"
)


folium.LayerControl().add_to(mapa)
mapa

### Se puede concluir que Trocafone tiende a tener más usuarios (por millón de habitantes) en los lugares más poblados de Brasil, por ejemplo en los estados de Sao Paulo, Rio de Janeiro, Bahia y Minas Gerais. Pero ocurre también que el segundo estado con más usuarios (por millón de habitantes) es el Distrito Federal, lo cual puede deberse a que es una ciudad desarollada tecnológicamente ya que es la capital brasilera.